In [5]:
import csv

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from models.TransformerEncoderDecoder_Traditional import TransformerModel

In [6]:
EPOCH_LOSSES_SAVEPATH = "./models/transformer_epoch_losses2.csv"
BATCH_LOSSES_SAVEPATH = "./models/transformer_batch_losses2.csv"

def save_list_to_csv(lst, filename):
    with open(filename, 'w', newline='') as file:
        wr = csv.writer(file)
        wr.writerow(lst)

def evaluate_batch_mean_average_accuracy(y_truth, y_pred, device):
    matches = [x == y for (x,y) in zip(y_truth, y_pred)]
    maas = []
    for batch in range(0, len(matches), 10):
        num_correct = 0
        summ = 0
        for i in range(0, 10):
            if matches[batch+i] == 1:
                num_correct += 1
                summ += (num_correct / (i+1)) / 10
        summ = torch.tensor(summ).to(device)
        maas.append(summ)
    return sum(maas) / len(maas)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = "./models/transformer_encoder_decoder_12.pt"

from data.SpotifyDataset import SpotifyDataset

train_set = SpotifyDataset("./data/train_data_20.csv", "./data/track_feats.csv")
val_set = SpotifyDataset("./data/val_data_20.csv", "./data/track_feats.csv")
test_set = SpotifyDataset("./data/test_data_20.csv", "./data/track_feats.csv")

datasets = {"train": train_set,
            "val": val_set,
            "test": test_set}

In [7]:
#len(train_set.shape)

### TRAINING BLOCK

In [8]:
nlabels = 2 # the size of vocabulary
n_input1 = 68 # embedding dimension
n_input2 = 30 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhid = 256
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value

model = TransformerModel(nlabels, n_input1, n_input2, nhead, nhid, nlayers, dropout=0.5).to(device)
loss_fn = nn.CrossEntropyLoss()

batch_size = 64
learning_rate = 1e-3
weight_decay = 2.5e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), weight_decay=weight_decay)

num_epochs = 3
best_maa = 0
batch_losses = []
epoch_losses = []
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    for phase in ['train', 'val']:
        print("Running {} phase...".format(phase))
        total_maa = []
        total_loss = []
        dataloader = DataLoader(datasets[phase], batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
        for i, (X_encode, X_decode, y) in enumerate(dataloader):
            if i > 50:
                break
            if i % 10 == 0:
                print("Calculating batch {} / {}".format(i, len(dataloader)))
            print(X_decode.shape, X_encode.shape, y.shape)
            X_encode, X_decode, y = X_encode.to(device), X_decode.to(device), y.to(device)
            if phase == 'train':
                model.train()
#                 print(X_encode.shape)
#                 print(X_decode.shape)
                merged_input = torch.cat((X_encode, X_decode), 1).to(device)
                # reshape merged input to 20, 64, 68
                # currently, merged input size = 64, 20, 68
                merged_input = merged_input.permute(1, 0, 2).to(device)

                # calculate what trg is:
                # y = (64, 10, 1)
                # N - batch size
                # S - source sequence length
                # T - target sequence length
                # E - embedding size / feature number
                # (T,N,E) (10,64,68)
#                 start_token = torch.ones((64,10,68)).to(device)
                start_token = torch.ones((10,64,68)).to(device)
    
                
                print(merged_input.shape)
                print(start_token.shape)
                
                scores = model(merged_input, start_token).flatten(start_dim=0, end_dim=1)
                labels = y.flatten(start_dim=0, end_dim=1).squeeze()
                
                cross_ent_loss = loss_fn(scores, labels)
                y_pred = torch.argmax(scores, dim=1)
                maa = evaluate_batch_mean_average_accuracy(labels, y_pred, device)
                loss = cross_ent_loss - maa
                
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                total_loss.append(loss)
                batch_losses.append(loss.item())
            else:
                model.eval()
                scores = model(X_encode, X_decode).flatten(start_dim=0, end_dim=1)
                labels = y.flatten(start_dim=0, end_dim=1).squeeze()
                y_pred = torch.argmax(scores, dim=1)
                maa = evaluate_batch_mean_average_accuracy(labels, y_pred, device)
                total_maa.append(maa)
        if phase == 'train':
            epoch_loss = sum(total_loss) / len(total_loss)
            print("Epoch {} Avg. Loss: {}".format(epoch, epoch_loss))
            epoch_losses.append(epoch_loss.item())
        else:
            epoch_maa = sum(total_maa) / len(total_maa)
            print("Epoch {} Avg. MAA: {}, Best MAA: {}".format(epoch, epoch_maa, best_maa))
            if epoch_maa > best_maa:
                torch.save(model.state_dict(), MODEL_PATH)
                best_maa = epoch_maa
    print()
print()
print("List of avg. loss across epochs: ")
print(epoch_losses)
save_list_to_csv(epoch_losses, EPOCH_LOSSES_SAVEPATH)
print("List of losses across batches: ")
print(batch_losses)
save_list_to_csv(batch_losses, BATCH_LOSSES_SAVEPATH)


Epoch 1/3
Running train phase...
Calculating batch 0 / 924
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([20, 64, 68])
torch.Size([10, 64, 68])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([20, 64, 68])
torch.Size([10, 64, 68])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([20, 64, 68])
torch.Size([10, 64, 68])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([20, 64, 68])
torch.Size([10, 64, 68])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([20, 64, 68])
torch.Size([10, 64, 68])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([20, 64, 68])
torch.Size([10, 64, 68])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([20, 64, 68])
torch.Size([10, 64, 68])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])

torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
Calculating batch 30 / 59
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([

torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([20, 64, 68])
torch.Size([10, 64, 68])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([20, 64, 68])
torch.Size([10, 64, 68])
Calculating batch 50 / 924
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([20, 64, 68])
torch.Size([10, 64, 68])
Epoch 1 Avg. Loss: 0.25970643758773804
Running val phase...
Calculating batch 0 / 59
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1]

Calculating batch 30 / 924
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([20, 64, 68])
torch.Size([10, 64, 68])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([20, 64, 68])
torch.Size([10, 64, 68])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([20, 64, 68])
torch.Size([10, 64, 68])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([20, 64, 68])
torch.Size([10, 64, 68])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([20, 64, 68])
torch.Size([10, 64, 68])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([20, 64, 68])
torch.Size([10, 64, 68])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([20, 64, 68])
torch.Size([10, 64, 68])
torch.Size([64, 10, 68]) torch.Size([64, 10, 68]) torch.Size([64, 10, 1])
torch.Size([20, 64, 68])
torch.

### TESTING BLOCK

In [11]:
test_model = TransformerModel(nlabels, n_input1, n_input2, nhead, nhid, nlayers, dropout=0.5).to(device)
test_model.load_state_dict(torch.load(MODEL_PATH))

with torch.no_grad():
    dataloader = DataLoader(datasets["test"], batch_size=batch_size, shuffle=False, num_workers=2)
    total_maa = []
    for i, (X_encode, X_decode, labels) in enumerate(dataloader):
        X_encode, X_decode, y = X_encode.to(device), X_decode.to(device), y.to(device)
        labels = y.flatten(start_dim=0, end_dim=1).squeeze()
        scores = test_model(X_encode, X_decode).flatten(start_dim=0, end_dim=1)
        y_pred = torch.argmax(scores, dim=1)
        print(y_pred)
        print(y_pred.shape)
        maa = evaluate_batch_mean_average_accuracy(labels, y_pred, device)
        total_maa.append(maa)
    print("Average batch MAA over test set: {}".format(sum(total_maa) / len(total_maa)))

tensor([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
        1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
        0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,

...

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,

tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,

tensor([0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,